General working environment for python:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Bio.Seq
import os, re
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "danchurchthomas@gmail.com"

Imports for the sequence viewer:

In [2]:
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

import jupyter_seqPlotters
## homemade module, code stolen from 
## https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

## Plants, PKS and NRPS primers

As a general indicator, how many records do we find in the NCBI nucleotide database for PKSes, type 1?

In [3]:
plantPKShandle = Entrez.esearch(db="nucleotide",
    term="(((Polyketide Synthase) AND Type I) NOT Type II) NOT Type III AND green plants[porgn:__txid33090]",
    idtype="acc",
    retmax=1000,
    usehistory="y",
)


In [5]:
record = Entrez.read(plantPKShandle)
webenv = record["WebEnv"]
query_key = record["QueryKey"]

In [7]:
record['Count']

'246'

So in the vast nucleotide database, we find 246 references to PKS I in plants...

In [9]:
fetch = Entrez.efetch(
    db="nucleotide",
    rettype="fasta",
    retmode="text",
    retmax=len(record["IdList"]),
    webenv=webenv,
    query_key=query_key,
    idtype="acc",
)

In [10]:
dataPlantPKS = fetch.read()

In [11]:
pksfile='/home/daniel/Documents/job_apps/panama/bigFilesBiodiv/plantPKS.fa'
with open(pksfile, 'w') as f:
    f.write(dataPlantPKS)
    fetch.close()

In [14]:
aln = AlignIO.read('testCombo.afa','fasta')
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)


Bokeh(Column)